In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model
from utils import create_dataset_from_one_image

In [ ]:
# Model/training hyperparameters
INPUT_SHAPE = (512, 512, 3)
CROPPING_MODE = 'independent_patches'  # (copy-paste a mode from: independent_patches, overlapped_patches, random_patches)
TRAIN_IMAGES_NUMBER = None # required only for 'random_patches' mode
PATCHES_STEP = None  # required only for 'overlapped_patches' mode
INITIAL_FILTERS = 32
BATCH_SIZE = 8
EPOCHS = 150

In [ ]:
# Load the initial input and output images and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)

x_initial_train = plt.imread('train/101-INPUT.jpg').astype('float32')[650:-550]
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')[650:-550]
x_initial_valid = plt.imread('validation/201-INPUT.jpg').astype('float32') [500:-50]
y_initial_valid = plt.imread('validation/201-OUTPUT-GT.png')[500:-50]

In [ ]:
# Create a dataset with the selected cropping mode
x_train, y_train = create_dataset_from_one_image(x_initial_train, 
                                                 y_initial_train, 
                                                 INPUT_SHAPE[0], 
                                                 mode=CROPPING_MODE, 
                                                 new_images_number=TRAIN_IMAGES_NUMBER, 
                                                 patches_step=PATCHES_STEP)

training_images_number = x_train.shape[0]

In [ ]:
# create a validation dataset
x_valid, y_valid = create_dataset_from_one_image(x_initial_valid, 
                                                 y_initial_valid, 
                                                 INPUT_SHAPE[0], 
                                                 mode='independent_patches')

# select a subset from the validation dataset equal to 33% of the training dataset
val_data_size = training_images_number // 3
perm = np.random.permutation(training_images_number)
x_valid = x_valid[perm][:val_data_size]
y_valid = y_valid[perm][:val_data_size]

print(x_valid.shape)

In [ ]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [ ]:
BATCH_SIZE = 8

# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [ ]:
training_images_number = x_train.shape[0]
steps_per_epoch = training_images_number // BATCH_SIZE

# Define the model and train
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS)
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_generator, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS,
                    validation_data=(x_valid, y_valid)
                   )

In [ ]:
parent_path = 'models/f001_w_val_data'

if not os.path.exists(parent_path):
    os.makedirs(parent_path)

# save the model and history of the training (loss, accuracy)
model_name = f'unet_inputsize{INPUT_SHAPE[0]}_initialfilters{INITIAL_FILTERS}_initialimagesnumber{x_train.shape[0]}_{CROPPING_MODE}_optimizerAdam_lr{"{:0.4f}".format(model.optimizer.lr.value().numpy())}_epochs{EPOCHS}' #TODO: loss and metric'
model_path = os.path.join(parent_path, model_name)
model.save(model_path)
np.save(os.path.join(model_path, 'history'), history.history)